In [91]:
# imports
import drgmum
from drgmum.toolkit.dataset import SmartDataset
from drgmum.chemdata.fingerprints import molprint2d_count_fingerprinter
from drgmum.chemdata.chembl.sqlite_queries import get_smiles
from drgmum.chemdata.utils.pandas_utils import two_level_series_to_csr
import logging
logging.basicConfig()
import os
data_folder = '/var/data/users/local/pocha/data'
chembl21db = os.path.join(data_folder, 'DRGMUM/chembl_21.db')
chembl_ids_file = os.path.join(data_folder, 'SCFP/Random_compounds_100.sdf')
similarity_matrix_file = os.path.join(data_folder, 'SCFP/Similarity150Dawid.csv')
folder_with_pairs = os.path.join(data_folder, 'SCFP/pairs')
from load_constraints import get_chembls, load_similarity_matrices, get_mapping
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.cluster import KMeans
from sklearn.utils.estimator_checks import check_estimator
from sklearn.model_selection import train_test_split
random_state = 666

In [86]:
# load data

# 1. load chembl IDs
chembl_ids = get_chembls(chembl_ids_file)
smiles = get_smiles(chembl21db, chembl_ids)

# 2. load fingerprints
a = molprint2d_count_fingerprinter(smiles) # nieważne
sprase_fp, row_labels, col_labels = two_level_series_to_csr(a)

# 3. load constraints
bin_sim, scale_sim, mapping_idx_chembl = load_similarity_matrices(similarity_matrix_file, chembl_ids_file, folder_with_pairs)
bin_sim_list = zip(zip(*bin_sim.nonzero()), bin_sim.data)
scale_sim_list = zip(zip(*scale_sim.nonzero()), scale_sim.data)

# 4. remove contraints that are duplicate (sim(x,y)=sim(y,x)) or
bin_sim_list = [x for x in bin_sim_list if x[0][0]<x[0][1]]
scale_sim_list = [x for x in scale_sim_list if x[0][0]<x[0][1]]

# 5. podział więzów na foldy
y_train, y_test = train_test_split(bin_sim_list, train_size=0.8, random_state=random_state)
# 343 - train, 86 - test 

INFO:drgmum.chemdata.chembl.sqlite_queries:Extracting SMILES from the database...
INFO:drgmum.chemdata.chembl.sqlite_queries:Done
INFO:drgmum.chemdata.fingerprints._openbabel:Processing 100 SMILES...
100it [00:00, 321402.61it/s]
645it [00:00, 479923.02it/s]
100%|██████████| 2144/2144 [00:00<00:00, 312535.63it/s]

21 pairs were omitted


In [92]:
# chcemy by mapowanie indeks-chembl dla sparse_fp i dla bin/scale_sim było takie samo
constraints_mapping = get_mapping(all_compounds_file=chembl_ids_file)
print constraints_mapping

bidict({0: 'CHEMBL426317', 1: 'CHEMBL261489', 2: 'CHEMBL201454', 3: 'CHEMBL475407', 4: 'CHEMBL273173', 5: 'CHEMBL567791', 6: 'CHEMBL300835', 7: 'CHEMBL299498', 8: 'CHEMBL428879', 9: 'CHEMBL598443', 10: 'CHEMBL289436', 11: 'CHEMBL59584', 12: 'CHEMBL255802', 13: 'CHEMBL1629810', 14: 'CHEMBL1278202', 15: 'CHEMBL472789', 16: 'CHEMBL223468', 17: 'CHEMBL261490', 18: 'CHEMBL534', 19: 'CHEMBL263881', 20: 'CHEMBL19215', 21: 'CHEMBL55914', 22: 'CHEMBL475778', 23: 'CHEMBL2059590', 24: 'CHEMBL41389', 25: 'CHEMBL629', 26: 'CHEMBL371300', 27: 'CHEMBL53308', 28: 'CHEMBL305275', 29: 'CHEMBL220900', 30: 'CHEMBL215468', 31: 'CHEMBL2087337', 32: 'CHEMBL201930', 33: 'CHEMBL14563', 34: 'CHEMBL76307', 35: 'CHEMBL590078', 36: 'CHEMBL157138', 37: 'CHEMBL42', 38: 'CHEMBL206845', 39: 'CHEMBL590483', 40: 'CHEMBL297784', 41: 'CHEMBL88894', 42: 'CHEMBL2323579', 43: 'CHEMBL1214830', 44: 'CHEMBL220207', 45: 'CHEMBL76301', 46: 'CHEMBL2110371', 47: 'CHEMBL280465', 48: 'CHEMBL417082', 49: 'CHEMBL209324', 50: 'CHEMBL121

In [ ]:
# define score
def my_score(labels_true, labels_pred):
    # labels true: ((x_i, x_j), sim(x_i, x_j))
    # labels_pred: list
    true, pred = np.array(labels_true), np.array(labels_pred)
    

In [76]:
# define model (this is rather not needed)
# class KMeans_SCFP(KMeans):
#     def score():
#         pass
    
# check_estimator(KMeans_SCFP)

In [ ]:
# define models and hiperparameters



In [ ]:
# define experiment

results = {}
for hips in hiperparameters:
    scores = []
    for i in range(5):
        model = KMeans(**hips)
        model.fit(X)
        predictions = model.predict(X)
        scores.append(scoring(y_train, predictions))
    results[str(hips)]=np.mean(scores)